In [2]:
import os
import torch
from torch import nn
from torch.utils.data import DataLoader
from torchvision import datasets, transforms

#使用算力更高的GPU
device = "cuda" if torch.cuda.is_available() else "cpu"
#自定义神经网络
class NeuralNetwork(nn.Module):
    def __init__(self):
        super(NeuralNetwork, self).__init__() #多重继承的问题，调用时，先把类NeuralNetwork的对象转换为父类的对象
        self.flatten = nn.Flatten() #从第一个维度展平到最后一个维度
        self.linear_relu_stack = nn.Sequential( #序列容器
            nn.Linear(28*28, 512), #完成从输入层到隐藏层的线性变换
            nn.ReLU(), #非线性激活函数类，可自选
            nn.Linear(512, 512),
            nn.ReLU(),
            nn.Linear(512, 10), #完成从隐藏层到输出层的线性变换
        )

    def forward(self, x): #层层推进
        x = self.flatten(x)
        logits = self.linear_relu_stack(x)
        return logits
#实例化
model = NeuralNetwork().to(device)
print(model)
X = torch.rand(1, 28, 28, device=device)
logits = model(X)   #传入一个28*28的图形张量，输出为[1,10]的张量
#分类函数将多个神经元输出映射到（0，1）区间，可看作概率，从而进行多分类
pred_probab = nn.Softmax(dim=1)(logits) #对logits进行缩放，使得在维度1上的元素相加等于1，即10列元素相加等于1
y_pred = pred_probab.argmax(1) #返回指定维度1最大值的序号，即找出最大数所在列
print(f"Predicted class: {y_pred}")
print(pred_probab.shape)

NeuralNetwork(
  (flatten): Flatten(start_dim=1, end_dim=-1)
  (linear_relu_stack): Sequential(
    (0): Linear(in_features=784, out_features=512, bias=True)
    (1): ReLU()
    (2): Linear(in_features=512, out_features=512, bias=True)
    (3): ReLU()
    (4): Linear(in_features=512, out_features=10, bias=True)
  )
)
Predicted class: tensor([6], device='cuda:0')
torch.Size([1, 10])


In [3]:
#输入
input_image = torch.rand(3,28,28)
print(input_image.size())
#扁平化，将2D 28x28 图像转换为 784 像素值的连续数组
flatten = nn.Flatten()
flat_image = flatten(input_image)
print(flat_image.size())
#线性层，是一个模块，使用其存储的权重和偏差对输入线性变换
layer1 = nn.Linear(in_features=28*28, out_features=20)
hidden1 = layer1(flat_image)
print(hidden1.size())
#分类函数非线性激活，输入输出间的映射
print(f"Before ReLU: {hidden1}\n\n")
hidden1 = nn.ReLU()(hidden1)
print(f"After ReLU: {hidden1}")
#序列容器
seq_modules = nn.Sequential(
    flatten,
    layer1,
    nn.ReLU(),
    nn.Linear(20, 10)
)
input_image = torch.rand(3,28,28)
logits = seq_modules(input_image)
#SoftMax层,对数按比例缩放为值[0,1]表示模型对每个类的预测概率
softmax = nn.Softmax(dim=1)
pred_probab = softmax(logits)

#模型参数
print(f"Model structure: {model}\n\n") #模型结构
for name, param in model.named_parameters():
    print(f"Layer: {name} | Size: {param.size()} | Values : {param[:2]} \n")

torch.Size([3, 28, 28])
torch.Size([3, 784])
torch.Size([3, 20])
Before ReLU: tensor([[ 0.5880,  0.0643,  0.2548, -0.6564,  0.0170, -0.5837,  0.0762,  0.1946,
          0.1768, -0.1685,  0.4129,  0.1765, -0.2400, -0.0649,  0.1474,  0.1023,
         -0.0567,  0.2518,  0.3235,  0.5008],
        [ 0.3247, -0.3103,  0.6077, -0.4354,  0.0078, -0.5167, -0.4080, -0.0459,
          0.2121,  0.0440, -0.1382, -0.0493, -0.5178, -0.3627,  0.2145,  0.4915,
         -0.0425,  0.1138, -0.0413,  0.1356],
        [ 0.3093,  0.1840,  0.9382, -0.6254, -0.2982, -0.1488, -0.2883, -0.0528,
          0.0321, -0.1231,  0.0239, -0.0826, -0.4837, -0.1022,  0.4656,  0.2779,
         -0.3499, -0.0411,  0.0334,  0.0844]], grad_fn=<AddmmBackward0>)


After ReLU: tensor([[0.5880, 0.0643, 0.2548, 0.0000, 0.0170, 0.0000, 0.0762, 0.1946, 0.1768,
         0.0000, 0.4129, 0.1765, 0.0000, 0.0000, 0.1474, 0.1023, 0.0000, 0.2518,
         0.3235, 0.5008],
        [0.3247, 0.0000, 0.6077, 0.0000, 0.0078, 0.0000, 0.0000, 0.00